In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [2]:
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return text, label

In [3]:
import numpy as np

# 70で作成した行列をロード
X_train = np.load("./matrix/x_train.npy")
Y_train = np.load("./matrix/y_train.npy")

In [4]:
X_train_tensor = torch.from_numpy(X_train)
Y_train_tensor =torch.from_numpy(Y_train)
datasets = TextDataset(X_train, Y_train)
train_dataloader = DataLoader(datasets, shuffle=True, batch_size=64)

In [5]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(300, 4),
            nn.Softmax()
        )

    def forward(self, x):
        logits = self.layer(x)
        return logits

In [6]:
model = Model()
learning_rate = 1e-3
batch_size = 64
epochs = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 予測と損失の計算
        X = X.float()
        pred = model.forward(X)
        loss = loss_fn(pred, y.to(torch.float64))
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
print("Before training!\n ------------------------------")
for name, param in model.named_parameters():
    print(f"{name}: {param.data}")
    print(f"Shape: {param.shape}")
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
print("Done training!\n ------------------------------")
for name, param in model.named_parameters():
    print(f"{name}: {param.data}")
    print(f"Shape: {param.shape}")

Before training!
 ------------------------------
layer.0.weight: tensor([[-0.0214,  0.0534, -0.0024,  ...,  0.0211,  0.0013, -0.0283],
        [-0.0393, -0.0359,  0.0102,  ...,  0.0401, -0.0213, -0.0289],
        [-0.0488,  0.0452, -0.0263,  ..., -0.0148, -0.0295, -0.0283],
        [-0.0042, -0.0317, -0.0575,  ...,  0.0374, -0.0153,  0.0562]])
Shape: torch.Size([4, 300])
layer.0.bias: tensor([-0.0447, -0.0495, -0.0144, -0.0132])
Shape: torch.Size([4])
Epoch 1
-------------------------------
loss: 1.386075  [    0/10671]
loss: 1.388726  [ 6400/10671]
Epoch 2
-------------------------------
loss: 1.385452  [    0/10671]
loss: 1.384689  [ 6400/10671]


/Users/tajirimanato/.pyenv/versions/3.12.2/lib/python3.12/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch 3
-------------------------------
loss: 1.382439  [    0/10671]
loss: 1.384349  [ 6400/10671]
Epoch 4
-------------------------------
loss: 1.383568  [    0/10671]
loss: 1.379933  [ 6400/10671]
Epoch 5
-------------------------------
loss: 1.380964  [    0/10671]
loss: 1.379485  [ 6400/10671]
Epoch 6
-------------------------------
loss: 1.378590  [    0/10671]
loss: 1.379326  [ 6400/10671]
Epoch 7
-------------------------------
loss: 1.373747  [    0/10671]
loss: 1.372499  [ 6400/10671]
Epoch 8
-------------------------------
loss: 1.373047  [    0/10671]
loss: 1.373729  [ 6400/10671]
Epoch 9
-------------------------------
loss: 1.367910  [    0/10671]
loss: 1.372517  [ 6400/10671]
Epoch 10
-------------------------------
loss: 1.371049  [    0/10671]
loss: 1.373028  [ 6400/10671]
Done training!
 ------------------------------
layer.0.weight: tensor([[-0.0226,  0.0556, -0.0036,  ...,  0.0252,  0.0061, -0.0338],
        [-0.0405, -0.0375,  0.0130,  ...,  0.0404, -0.0248, -0.027

In [9]:
torch.save(model, 'model_73.pth')